# Transformaciones y acciones

In [178]:
from pyspark import SparkContext
from pyspark import rdd
from operator import add
from pyspark.sql.types import (StructType, 
                               StructField,
                               IntegerType,
                               StringType,
                               FloatType,
                               Row
                               )
from pyspark.sql import SQLContext

In [179]:
# Crear contexto
sc = SparkContext(
    master="local",
    appName="estructuraRDD"
    )
# sqlContext = SQLContext(sc)

In [180]:
# Crear un RDD
rdd1 = sc.parallelize([1,2,3])
print("Type: ",type(rdd1))
##  Visualizar RDD
rdd1.collect()

Type:  <class 'pyspark.rdd.RDD'>


[1, 2, 3]

In [181]:
# ver proceso en SPARK
sc

<SparkContext master=local appName=estructuraRDD>

In [182]:
# PATH de archivos
path =  "/home/luis/Documents/Codes/Python_Spark/files/Olimpiadas/"

In [183]:
# Cargar archivos con RDD

equiposOlimpicosRDD = sc.textFile(path+"paises.csv").map(
    lambda line: line.split(",")
)
def cargaDeArchivos(file:str)->rdd:
    archivo = sc.textFile(path+file).map(
        lambda line: line.split(",")
        )
    return archivo

equiposOlimpicosRDD = cargaDeArchivos("paises.csv")

# Visualizar RDD
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [184]:
# Contar cantidad depaises distintos (columna 3)
equiposOlimpicosRDD.map(
    lambda x: (x[2])
).distinct().count()

231

In [185]:
# Contar equipo por pais (agrupando)
equiposOlimpicosRDD.map(
    lambda x: (x[2],x[1])
).groupByKey().mapValues(len).take(10)

[('sigla', 1),
 ('AUT', 11),
 ('MEX', 9),
 ('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('SUI', 17),
 ('ALB', 1),
 ('POR', 21),
 ('FRA', 155)]

In [186]:
# Analisar solo un equipo (filtrando)
equipoasArgentinos = equiposOlimpicosRDD.filter(
    lambda x: "ARG" in x
)
equipoasArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [187]:
# Validar cantidad de datos antes de usar el collect
equipoasArgentinos.count() 

18

In [188]:
# contar por tiempo (milisegundos)
equiposOlimpicosRDD.countApprox(20) # 20 milisegundos

1185

In [189]:
# Cargar archivo deportistas.csv y deportistas 2
deportistaOlimpicoRDD = cargaDeArchivos("deportista.csv")
deportistaOlimpicoRDD2 = cargaDeArchivos("deportista2.csv")

# Unir ambas bases de datos de deportistas
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)
deportistaOlimpicoRDD.count()

135572

In [190]:
# Ver cabeceras de los archivos
print(equiposOlimpicosRDD.top(1))
print(deportistaOlimpicoRDD.top(1))

[['id', 'equipo', 'sigla']]
[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id']]


In [191]:
# Unir datos de deportistas con equipos
deportistaOlimpicoRDD.map(lambda x: [x[-1],x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda y: [y[0],y[2]])
).takeSample(False,15,25)

[('1096',
  (['18804', 'Kellogg Kennon Venable Casey', '1', '30', '0', '0'], 'USA')),
 ('624', (['13109', 'Harry Arthur Bonavia', '1', '20', '0', '0'], 'MLT')),
 ('576', (['32099', 'Assaf ElMurr', '1', '24', '0', '0'], 'LIB')),
 ('1178', (['97638', 'Oto Pustoslemek', '1', '20', '0', '0'], 'YUG')),
 ('399', (['129142', 'Marc Weber', '1', '24', '191', '91'], 'GER')),
 ('514', (['118314', 'Asuna Tanaka', '2', '24', '164', '52'], 'JPN')),
 ('273',
  (['56134', 'Niels Christian Kold Jrgensen', '1', '21', '0', '0'], 'DEN')),
 ('198', (['129714', 'Claudio Wernli', '1', '21', '172', '68'], 'CHI')),
 ('362',
  (['14043', 'Julien Joseph Jean Boullery', '1', '25', '0', '0'], 'FRA')),
 ('770', (['104143', 'Mamoon Sadiq', '1', '23', '0', '0'], 'PAK')),
 ('199', (['110045', 'Shu Siyao', '2', '23', '167', '52'], 'CHN')),
 ('514', (['132430', 'Atsushi Yanagisawa', '1', '23', '177', '72'], 'JPN')),
 ('507', (['135386', 'Enea Zuffi', '1', '20', '0', '0'], 'ITA')),
 ('810',
  (['8368', 'Edmund Ludwik Bar

In [192]:
# cargar el resultado de los deportistas
resultadosOlimpicos = cargaDeArchivos("resultados.csv")
resultadosOlimpicos.takeSample(False,10,27)

[['16847', 'Silver', '9033', '19', '3'],
 ['266698', 'NA', '133415', '45', '253'],
 ['163852', 'NA', '82236', '31', '198'],
 ['128275', 'NA', '64585', '7', '325'],
 ['24154', 'NA', '12669', '36', '39'],
 ['38532', 'NA', '19819', '29', '43'],
 ['176631', 'NA', '88816', '30', '370'],
 ['102605', 'NA', '51907', '51', '110'],
 ['216052', 'NA', '108448', '23', '110'],
 ['214964', 'NA', '107907', '7', '98']]

In [193]:
# Eliminar deportistas que no ganaron medalla
resultadosOlimpicosGanadores = resultadosOlimpicos.filter(
    lambda l: l[1] != "NA"
)

In [194]:
resultadosOlimpicosGanadores.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [195]:
# Poner el pais de origen a los deportistas
deportistaPaises = deportistaOlimpicoRDD.map(
        lambda x: [x[-1],x[:-1]]
    ).join(
        equiposOlimpicosRDD.map(
            lambda y: [y[0],y[2]]
        )
    )
deportistaPaises.take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [196]:
# filtrar solo por ganadores de medallas
paisesMedallas=deportistaPaises.join(resultadosOlimpicosGanadores)
paisesMedallas.take(5)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold'))]

In [197]:
valoresMedallas = {
    'Gold': 7,
    'Silver': 4,
    'Bronze':3
}

In [198]:
paisesMedallas = paisesMedallas.map(
    lambda x: 
        (x[1][0][-1],
        valoresMedallas[x[1][1]])
     )
paisesMedallas.groupByKey().mapValues(sum).take(10)

[('ARG', 12519),
 ('FIN', 33),
 ('USA', 1319),
 ('GRE', 182),
 ('IRL', 14),
 ('STP', 39),
 ('BHU', 147),
 ('CAN', 32427),
 ('ANZ', 129),
 ('THA', 14)]

In [199]:
paisesMedallas.reduceByKey((add)).sortBy(
    lambda x: x[1],ascending=False
    ).take(10)

[('CAN', 32427),
 ('ARG', 12519),
 ('HUN', 8145),
 ('MEX', 4593),
 ('BLR', 2864),
 ('RSA', 2841),
 ('USA', 1319),
 ('LTU', 1228),
 ('AZE', 1218),
 ('MGL', 1168)]

In [200]:
sc

<SparkContext master=local appName=estructuraRDD>

In [201]:
# Cerrar sesion (importante hacerlo)
sc.stop()